In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm
import pandas as pd
import requests

In [16]:
url = 'https://sport.tv2.dk/seneste'
driver = webdriver.Chrome()
driver.get(url)

headers = {
    'Name': 'Nikolaj Fu Jie Lin', 
    'E-mail':'qhs970@alumni.ku.dk',
    'Message': 'Scraping as a part of our exam project at UCPH, Intoduction to social data science' 
}
requests.get(url, headers)
time.sleep(5)

In [17]:
cookie = driver.find_element(By.CSS_SELECTOR, '#onetrust-reject-all-handler')
cookie.click()
time.sleep(5)

In [33]:
Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
Vis_flere.click()
time.sleep(5)

In [5]:
"""
i = 0
while i <= 10:
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
        Vis_flere.click()
        time.sleep(1)
        i = i + 1
    except:
        time.sleep(1)
"""

"\ni = 0\nwhile i <= 10:\n    try:\n        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')\n        Vis_flere.click()\n        time.sleep(1)\n        i = i + 1\n    except:\n        time.sleep(1)\n"

In [79]:
"""
i = 0
count_of_one_column_lists = 0
while i <= 10:
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
        Vis_flere.click()

        new_count_of_one_column_lists = 0
        while new_count_of_one_column_lists <= count_of_one_column_lists:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
            time.sleep(0.25)

        count_of_one_column_lists = new_count_of_one_column_lists
        i += 1
    except:
        time.sleep(1)
"""

"\ni = 0\ncount_of_one_column_lists = 0\nwhile i <= 10:\n    try:\n        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')\n        Vis_flere.click()\n\n        new_count_of_one_column_lists = 0\n        while new_count_of_one_column_lists <= count_of_one_column_lists:\n            soup = BeautifulSoup(driver.page_source, 'html.parser')\n            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))\n            time.sleep(0.25)\n\n        count_of_one_column_lists = new_count_of_one_column_lists\n        i += 1\n    except:\n        time.sleep(1)\n"

In [18]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

Headline = []
Category = []
Authors = []
Time = []
Live = []
All_text = []
nummer=0

Live_find1 = False
Live_find2 = False

max_retries = 10
retries = 0

Mathias = 'Ja'

max_iter = 15
i = 0
count_of_one_column_lists = 0
if Mathias == 'Nej':
    
    pbar1 = tqdm(total=max_iter, desc="Loading content")

    while i < 93:
        try:
            Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
            Vis_flere.click()

            new_count_of_one_column_lists = 0
            while new_count_of_one_column_lists <= count_of_one_column_lists:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
                time.sleep(0.25)

            count_of_one_column_lists = new_count_of_one_column_lists
            i += 1
            pbar1.update(1)
        except:
            time.sleep(1)
    pbar1.close()

pbar2 = tqdm(total=max_iter, desc="Scraping")
i = 0
for i in range(0, 15, 1):
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
        Vis_flere.click()

        new_count_of_one_column_lists = 0
        while new_count_of_one_column_lists <= count_of_one_column_lists:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
            time.sleep(0.25)

        count_of_one_column_lists = new_count_of_one_column_lists
    except:
        time.sleep(1)
        
    retries = 0
    while retries < max_retries:
        try:
            one_column_lists = soup.find_all('ul', {'data-columns': '1'})[i]
            break
        except:
            #print(i)
            retries = retries + 1
            time.sleep(1)
    
    if retries == max_retries:
        print(f"Max tries at: Number {nummer} and i: {i}")
        continue
        
    pbar3 = tqdm(total=len(soup.find_all('ul', {'data-columns': '1'})[i].find_all(class_='tc_teaser')), desc="Articles", leave=False)

    for one_column_list in one_column_lists:
        artikler = one_column_list.find_all(class_='tc_teaser')
            
        for artikel in artikler:
            nummer += 1

            # Initializations
            current_headline = None
            current_author = None
            current_time = None
            current_category = None
            current_text = []
            current_live = 0

            # Check for Live label
            if artikel.find('span', class_='tc_label tc_label--color-live'):
                Live_find1 = True
            else:
                Live_find1 = False

            # Extract link
            href = artikel.find('a', class_='tc_teaser__link').get('href')

            if href:
                driver.execute_script(f'window.open("{href}", "_blank");')
                driver.switch_to.window(driver.window_handles[-1])

                new_html = driver.page_source
                new_soup = BeautifulSoup(new_html, 'lxml')

                # Live blog check
                liveblog_body = new_soup.find('div', class_="tc_liveblog__body", attrs={'data-adobe-context': "article-body"})
                Live_find2 = bool(liveblog_body and liveblog_body.find('em') and liveblog_body.find('em').text == 'Denne liveblog opdateres ikke længere.')

                # Extract details
                if Live_find1 or Live_find2:
                    current_headline = new_soup.find('header', class_="tc_page__header").find('h1').text

                byline = new_soup.find('div', class_="tc_byline")
                if byline:
                    current_author = byline.find('strong').text
                timestamp = new_soup.find('time', class_='tc_timestamp')
                if timestamp:
                    current_time = timestamp.get('datetime')

                if liveblog_body:
                    article_body_div = liveblog_body
                else:
                    article_body_div = new_soup.find('div', class_='tc_page__body', attrs={'data-adobe-context': 'article-body'})

                if article_body_div:
                    rich_content_paragraphs = article_body_div.find_all('p')
                    for paragraph in rich_content_paragraphs:
                        current_text.append(paragraph.text)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

            if Live_find1 or Live_find2:
                current_live = 1
            else:
                current_headline = artikel.find('a', class_='tc_teaser__link').get('aria-label')


            if all([current_headline, current_author, current_time, current_text]):
                Headline.append(current_headline)
                Authors.append(current_author)
                Time.append(current_time)
                All_text.append(current_text)
                Live.append(current_live)
            else:
                print(f"Skipped article at nummer: {nummer} due to missing details: {href}")

            pbar3.update(1)
    pbar3.close()
    pbar2.update(1)
pbar2.close()
time.sleep(5)

Scraping:   0%|          | 0/15 [00:00<?, ?it/s]

Articles:   0%|          | 0/5 [00:00<?, ?it/s]

Articles:   0%|          | 0/5 [00:00<?, ?it/s]

Articles:   0%|          | 0/19 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

...

Skipped article at nummer: 425 due to missing details: https://nyheder.tv2.dk/samfund/2023-08-04-danske-kunder-foeler-sig-snydt-3760-kroner-for-aftale-der-aldrig-er-indgaaet


Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

In [1]:
print(len(Category), Category[0])
print(len(Authors), Authors[0])
print(len(Headline), Headline[0])
print(len(Time), Time[0])
print(len(Live), Live[0])
print(len(All_text), All_text[0])
time.sleep(5)

NameError: name 'Category' is not defined

In [25]:
data = {
    #'Category': Category,
    'Authors': Authors,
    'Headline': Headline,
    'Time': Time,
    'Live': Live,
    'All_text': All_text
}
df = pd.DataFrame(data)
time.sleep(5)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
df['Category'] = 'Sport'
#sorted_df = df.sort_values(by='Time')
#sorted_df

In [27]:
df.to_csv('sport.csv', index=False)
time.sleep(5)

In [26]:
df

,Authors,Headline,Time,Live,All_text,Category
0,Christian Kjær,Quick-Step præsenterer stjerne,2023-08-16T08:56:04.222Z,0,[Mikel Landa har skrevet en toårig kontrakt me...,Sport
1,Nicolai Dvinge,Messi skriver historie med Miami – selv modsta...,2023-08-16T08:18:43.961Z,0,[Lionel Messi har nu scoret ni mål i seks Inte...,Sport
2,Christian Kraglund,"Her mangler Wozniacki stadig noget, siger eksp...",2023-08-16T07:37:27.238Z,0,[Det kørte ikke på skinner for Caroline Woznia...,Sport
3,Lars Bruun-Mortensen,Skjern skal have ny træner – ekspert er overra...,2023-08-16T07:00:59.070Z,0,[Henrik Kronborg ønsker ikke at forlænge sin k...,Sport
4,Ritzau,Rune skal møde amerikansk wildcard-spiller,2023-08-16T06:05:28.567Z,0,[Mackenzie McDonald vandt sin kamp og brager i...,Sport
...,...,...,...,...,...,...
623,Oliver Orup Kristensen,"- Jeg var helt nede i sækken, erkender Axelsen",2023-07-28T10:27:19.608Z,0,[29-årige Axelsen skal møde japanske Kodai Nar...,Sport
624,Jonas Kjærsgaard,Danmark tæt på VM-overraskelse,2023-07-28T10:25:01.649Z,0,[Landsholdet har tre point efter to kampe ved ...,Sport
625,Simon Mølgaard,Vingegaards far: - Vi kan næsten ikke være i o...,2023-07-28T09:35:13.818Z,0,[Jonas Vingegaards far Claus Vingegaard er sto...,Sport
626,Jonas Kjærsgaard,Axelsen overvinder store problemer,2023-07-28T09:17:54.533Z,0,"[Kampen blev afgjort i tre sæt., Semifinalerne...",Sport


In [84]:
df.to_csv(r'C:\Users\Mathias\Desktop\Privat\Uni\~Kandidat\ISDS\Data.csv', index=False)
time.sleep(5)

In [17]:
import os
os.system("shutdown /s /t 10")

0

In [18]:
#os.system("shutdown /a")

0